In [16]:
import pandas as pd
import math
import numpy as np
from keras import Sequential
from keras.layers import Embedding, Flatten, Dense, Reshape
%matplotlib inline

# Load & prep data

## Read in data

In [2]:
!ls ~/Repositories/datasets/crimedata

communities.data.csv  communities.names.txt


In [166]:
# http://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.data
PATH = '~/Repositories/datasets/crimedata/communities.data.csv'

columns = ['state', 'county', 'community', 'communityname', 'fold', 'population', 'householdsize',
           'racepctblack', 'racePctWhite', 'racePctAsian', 'racePctHisp', 'agePct12t21', 'agePct12t29',
           'agePct16t24', 'agePct65up', 'numbUrban', 'pctUrban', 'medIncome', 'pctWWage', 'pctWFarmSelf', 
           'pctWInvInc', 'pctWSocSec', 'pctWPubAsst', 'pctWRetire', 'medFamInc', 'perCapInc', 'whitePerCap',
           'blackPerCap', 'indianPerCap', 'AsianPerCap', 'OtherPerCap', 'HispPerCap', 'NumUnderPov',
           'PctPopUnderPov', 'PctLess9thGrade', 'PctNotHSGrad', 'PctBSorMore', 'PctUnemployed', 'PctEmploy', 
           'PctEmplManu', 'PctEmplProfServ', 'PctOccupManu', 'PctOccupMgmtProf', 'MalePctDivorce', 'MalePctNevMarr', 
           'FemalePctDiv', 'TotalPctDiv', 'PersPerFam', 'PctFam2Par', 'PctKids2Par', 'PctYoungKids2Par',
           'PctTeen2Par', 'PctWorkMomYoungKids', 'PctWorkMom', 'NumIlleg', 'PctIlleg', 'NumImmig', 'PctImmigRecent',
           'PctImmigRec5', 'PctImmigRec8', 'PctImmigRec10', 'PctRecentImmig', 'PctRecImmig5', 'PctRecImmig8',
           'PctRecImmig10', 'PctSpeakEnglOnly', 'PctNotSpeakEnglWell', 'PctLargHouseFam', 'PctLargHouseOccup', 
           'PersPerOccupHous', 'PersPerOwnOccHous', 'PersPerRentOccHous', 'PctPersOwnOccup', 'PctPersDenseHous', 
           'PctHousLess3BR', 'MedNumBR', 'HousVacant', 'PctHousOccup', 'PctHousOwnOcc', 'PctVacantBoarded',
           'PctVacMore6Mos', 'MedYrHousBuilt', 'PctHousNoPhone', 'PctWOFullPlumb', 'OwnOccLowQuart', 'OwnOccMedVal',
           'OwnOccHiQuart', 'RentLowQ', 'RentMedian', 'RentHighQ', 'MedRent', 'MedRentPctHousInc', 'MedOwnCostPctInc',
           'MedOwnCostPctIncNoMtg', 'NumInShelters', 'NumStreet', 'PctForeignBorn', 'PctBornSameState',
           'PctSameHouse85', 'PctSameCity85', 'PctSameState85', 'LemasSwornFT', 'LemasSwFTPerPop', 'LemasSwFTFieldOps',
           'LemasSwFTFieldPerPop', 'LemasTotalReq', 'LemasTotReqPerPop', 'PolicReqPerOffic', 'PolicPerPop',
           'RacialMatchCommPol', 'PctPolicWhite', 'PctPolicBlack', 'PctPolicHisp', 'PctPolicAsian', 'PctPolicMinor',
           'OfficAssgnDrugUnits', 'NumKindsDrugsSeiz', 'PolicAveOTWorked', 'LandArea', 'PopDens', 'PctUsePubTrans',
           'PolicCars', 'PolicOperBudg', 'LemasPctPolicOnPatr', 'LemasGangUnitDeploy', 'LemasPctOfficDrugUn',
           'PolicBudgPerPop', 'ViolentCrimesPerPop',]

categorical_vars = ['state', 'fold', 'NumImmig', 'MedNumBR', 'MedYrHousBuilt', 'LemasSwornFT', 'LemasSwFTFieldOps',
                   'LemasTotalReq', 'OfficAssgnDrugUnits', 'NumKindsDrugsSeiz', 'PolicOperBudg', 'LemasGangUnitDeploy']

In [167]:
df = pd.read_csv(PATH, header=None, names=columns)

In [153]:
df.head()

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03


In [154]:
print("{} observations; {} features".format(len(df), len(df.columns)))

1994 observations; 128 features


## Do some cleanup and embedding prep

In [172]:
# replace the ?'s with NAN, drop the community name feature
df.replace(to_replace="?", value=np.NaN, inplace=True)
df.drop('communityname', axis=1, inplace=True)

# cast data types
for v in categorical_vars:
    # pandas will set categories without values (NULL or NaN) to -1.
    # this will cause problems later so here we're going to set nan to
    # a unique value so that pandas will create a new category for it
    df[v].fillna('M', inplace=True)
    df[v] = df[v].astype('category').cat.codes
for c in df.columns:
    if c not in categorical_vars:
        df[c] = df[c].astype('float32')
        df[c].fillna(0, inplace=True)

In [174]:
# verify we don't have any -1's in the dataset
df[categorical_vars].describe()

,state,fold,NumImmig,MedNumBR,MedYrHousBuilt,LemasSwornFT,LemasSwFTFieldOps,LemasTotalReq,OfficAssgnDrugUnits,NumKindsDrugsSeiz,PolicOperBudg,LemasGangUnitDeploy
count,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000
mean,21.877633,4.493982,2.675527,0.629388,23.697091,32.836008,32.849549,38.231194,26.220662,13.845537,32.896189,2.660983
std,13.249547,2.873694,5.919389,0.510363,11.153471,12.246248,3.762043,13.768759,8.990236,2.879798,12.103905,0.841958
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.000000,2.000000,0.000000,0.000000,17.000000,38.000000,34.000000,44.000000,30.000000,15.000000,38.000000,3.000000
50%,25.000000,4.000000,1.000000,1.000000,25.000000,38.000000,34.000000,44.000000,30.000000,15.000000,38.000000,3.000000
75%,33.000000,7.000000,2.000000,1.000000,32.000000,38.000000,34.000000,44.000000,30.000000,15.000000,38.000000,3.000000
max,45.000000,9.000000,46.000000,2.000000,48.000000,38.000000,34.000000,44.000000,30.000000,15.000000,38.000000,3.000000


In [122]:
target = 'ViolentCrimesPerPop'

# Train embeddings

Here we're going to train embeddings for each categorical variable we have, replacing these variables with the embeddings with the hopes having brought some more nuanced information to our training data which may have otherwise been missed in standard modeling.

In [9]:
# generate embed sizes
embed_sizes = []
for c in categorical_vars:
    len_unique = len(df[c].unique())
    if len_unique < 50:
        embed_dim = (len_unique, int(math.ceil(float(len_unique) / 2)))
        embed_sizes.append(embed_dim)
        print("{}: {} unique values; embed dim: {}".format(c, len_unique, embed_dim))

state: 46 unique values; embed dim: (46, 23)
fold: 10 unique values; embed dim: (10, 5)
NumImmig: 47 unique values; embed dim: (47, 24)
MedNumBR: 3 unique values; embed dim: (3, 2)
MedYrHousBuilt: 49 unique values; embed dim: (49, 25)
LemasSwornFT: 39 unique values; embed dim: (39, 20)
LemasSwFTFieldOps: 35 unique values; embed dim: (35, 18)
LemasTotalReq: 45 unique values; embed dim: (45, 23)
OfficAssgnDrugUnits: 31 unique values; embed dim: (31, 16)
NumKindsDrugsSeiz: 16 unique values; embed dim: (16, 8)
PolicOperBudg: 39 unique values; embed dim: (39, 20)
LemasGangUnitDeploy: 4 unique values; embed dim: (4, 2)


Here we'll manually create one to clarify and debug the process before iterating through all categorical variables and replacing them with the latent vector representations.

The steps taken here will be:

1. assign the training data consisting of only the categorical column of interest
2. define a simple model with the embedding of the dimensions defined earlier
3. train the model
4. extract the embedding weights from the trained model
5. replace the original categorical variable in the dataframe with the vector representation as features to the original training data

In [123]:
X_train = np.array(df['state']).reshape(len(df), 1)
X_train.shape

(1994, 1)

In [114]:
emb_state = Sequential()
emb_state.add(Embedding(46, 23, input_length=1))
emb_state.add(Flatten())
emb_state.add(Dense(1, activation='relu'))
emb_state.compile('rmsprop', 'mse')

In [115]:
emb_state.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_38 (Embedding)     (None, 1, 23)             1058      
_________________________________________________________________
flatten_1 (Flatten)          (None, 23)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 24        
Total params: 1,082
Trainable params: 1,082
Non-trainable params: 0
_________________________________________________________________


In [130]:
emb_state.fit(X_train, df[target].values, epochs=100, batch_size=128, verbose=1)

Epoch 1/100
1994/1994 [==============================] - 0s 20us/step - loss: 0.0425
Epoch 2/100
1994/1994 [==============================] - 0s 16us/step - loss: 0.0425
Epoch 3/100
1994/1994 [==============================] - 0s 14us/step - loss: 0.0425
Epoch 4/100
1994/1994 [==============================] - 0s 14us/step - loss: 0.0425
Epoch 5/100
1994/1994 [==============================] - 0s 14us/step - loss: 0.0426
Epoch 6/100
1994/1994 [==============================] - 0s 14us/step - loss: 0.0425
Epoch 7/100
1994/1994 [==============================] - 0s 15us/step - loss: 0.0424
Epoch 8/100
1994/1994 [==============================] - 0s 13us/step - loss: 0.0425
Epoch 9/100
1994/1994 [==============================] - 0s 13us/step - loss: 0.0425
Epoch 10/100
1994/1994 [==============================] - 0s 13us/step - loss: 0.0425
Epoch 11/100
1994/1994 [==============================] - 0s 14us/step - loss: 0.0425
Epoch 12/100
1994/1994 [==============================] - 0s 13

1994/1994 [==============================] - 0s 14us/step - loss: 0.0424
Epoch 97/100
1994/1994 [==============================] - 0s 15us/step - loss: 0.0424
Epoch 98/100
1994/1994 [==============================] - 0s 14us/step - loss: 0.0424
Epoch 99/100
1994/1994 [==============================] - 0s 13us/step - loss: 0.0424
Epoch 100/100
1994/1994 [==============================] - 0s 14us/step - loss: 0.0424


In [139]:
# and here is the vector representation for the first state category label
state_cat_lookup = emb_state.layers[0].get_weights()[0]
state_cat_lookup[0]

array([ 0.08285028,  0.05002696, -0.08325885, -0.04585622, -0.0082007 ,
       -0.02885346,  0.0287704 ,  0.0535865 , -0.00465109, -0.05954038,
       -0.01350796, -0.03775232,  0.06277085,  0.00424973, -0.07582595,
       -0.01416818, -0.02105121,  0.03722456, -0.00042511,  0.01504867,
       -0.07461265, -0.01139971,  0.02045437], dtype=float32)

Now we'll iterate through all categorical variables performing the above and completing the process by replacing the categorical variable within the dataframe with the relevant vector representation as additional features.

In [175]:
for i in range(len(categorical_vars)):
    # localize embed dim & category label
    embedding_dim = embed_sizes[i]
    column = categorical_vars[i]
    print("working on column: {}".format(column))
    
    # prep training data
    X_train = np.array(df[column]).reshape(len(df), 1)
    
    print("\tcreating embedding...")
    # create & train model
    emb = Sequential()
    emb.add(Embedding(embedding_dim[0], embedding_dim[1], input_length=1))
    emb.add(Flatten())
    emb.add(Dense(1, activation='relu'))
    emb.compile('rmsprop', 'mse')
    emb.fit(X_train, df[target].values, epochs=100, batch_size=128, verbose=0)
    
    # localize embedding weights
    cat_lookup = emb.layers[0].get_weights()[0]
    
    print("\treplacing column with {} latent features".format(cat_lookup.shape[1]))
    # add new feature columns
    for new_col_i in range(embedding_dim[1]):
        new_col_label = "_".join([column, str(new_col_i)])
        df[new_col_label] = df[column].apply(lambda x: cat_lookup[x][new_col_i])
    
    df.drop(column, axis=1, inplace=True)

working on column: state
	creating embedding...
	replacing column with 23 latent features
working on column: fold
	creating embedding...
	replacing column with 5 latent features
working on column: NumImmig
	creating embedding...
	replacing column with 24 latent features
working on column: MedNumBR
	creating embedding...
	replacing column with 2 latent features
working on column: MedYrHousBuilt
	creating embedding...
	replacing column with 25 latent features
working on column: LemasSwornFT
	creating embedding...
	replacing column with 20 latent features
working on column: LemasSwFTFieldOps
	creating embedding...
	replacing column with 18 latent features
working on column: LemasTotalReq
	creating embedding...
	replacing column with 23 latent features
working on column: OfficAssgnDrugUnits
	creating embedding...
	replacing column with 16 latent features
working on column: NumKindsDrugsSeiz
	creating embedding...
	replacing column with 8 latent features
working on column: PolicOperBudg
	cr

In [179]:
print(df.columns)
df.head()

Index([u'county', u'community', u'population', u'householdsize',
       u'racepctblack', u'racePctWhite', u'racePctAsian', u'racePctHisp',
       u'agePct12t21', u'agePct12t29',
       ...
       u'PolicOperBudg_12', u'PolicOperBudg_13', u'PolicOperBudg_14',
       u'PolicOperBudg_15', u'PolicOperBudg_16', u'PolicOperBudg_17',
       u'PolicOperBudg_18', u'PolicOperBudg_19', u'LemasGangUnitDeploy_0',
       u'LemasGangUnitDeploy_1'],
      dtype='object', length=301)


,county,community,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,...,PolicOperBudg_12,PolicOperBudg_13,PolicOperBudg_14,PolicOperBudg_15,PolicOperBudg_16,PolicOperBudg_17,PolicOperBudg_18,PolicOperBudg_19,LemasGangUnitDeploy_0,LemasGangUnitDeploy_1
0,0.0,0.0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,...,0.080219,-0.077829,-0.020429,-0.002106,0.026617,-0.069450,-0.019778,-0.074057,-0.186945,-0.149711
1,0.0,0.0,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,...,0.018194,-0.007709,-0.013976,-0.042676,0.014350,0.046788,-0.035263,0.012292,-0.050935,-0.006907
2,0.0,0.0,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,...,0.018194,-0.007709,-0.013976,-0.042676,0.014350,0.046788,-0.035263,0.012292,-0.050935,-0.006907
3,5.0,81440.0,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,...,0.018194,-0.007709,-0.013976,-0.042676,0.014350,0.046788,-0.035263,0.012292,-0.050935,-0.006907
4,95.0,6096.0,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,...,0.018194,-0.007709,-0.013976,-0.042676,0.014350,0.046788,-0.035263,0.012292,-0.050935,-0.006907


Now we have our embedding vectors added as feature columns in the dataframe and dropped the original category columns.

# Train & compare models

In [227]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Using embedding features

In [230]:
# prep training data and handle train/test split
X_train, X_test, y_train, y_test = train_test_split(df.drop(target, axis=1), df[target], test_size=0.2)

In [231]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("Random forest MSE: {}".format(mean_squared_error(y_test, y_pred)))

gbm = GradientBoostingRegressor()
gbm.fit(X_train, y_train)
y_pred = gbm.predict(X_test)

print("GBM MSE: {}".format(mean_squared_error(y_test, y_pred)))

Random forest MSE: 0.017415408534
GBM MSE: 0.0165254563109


## Excluding the embedding features

In [232]:
# prep training data and handle train/test split
continuous_vars = []
for c in columns:
    if c not in categorical_vars and c != 'communityname':
        continuous_vars.append(c)
        
X_train, X_test, y_train, y_test = train_test_split(df[continuous_vars].drop(target, axis=1), df[target], test_size=0.2)

In [233]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("Random forest MSE: {}".format(mean_squared_error(y_test, y_pred)))

gbm = GradientBoostingRegressor()
gbm.fit(X_train, y_train)
y_pred = gbm.predict(X_test)

print("GBM MSE: {}".format(mean_squared_error(y_test, y_pred)))

Random forest MSE: 0.0180950974896
GBM MSE: 0.0181914731775


Here we're seeing an improvement with the latent feature vectors.

| Model   | w/out latent features | w/ latent features |
|---------|-----------------------|--------------------|
| RF MSE  | 0.0180950974896       | 0.017415408534     |
| GBM MSE | 0.0181914731775       | 0.0165254563109    |
